In [276]:
import pandas as pd
import numpy as np

In [282]:
df_monday = pd.read_csv('monday.csv' ,sep= ';')
df_monday

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices
...,...,...,...
4879,2019-09-02 21:49:00,1442,checkout
4880,2019-09-02 21:49:00,1444,checkout
4881,2019-09-02 21:49:00,1445,dairy
4882,2019-09-02 21:50:00,1446,dairy


In [283]:
df_monday['timestamp']= pd.to_datetime(df_monday['timestamp'])
df_monday.set_index('timestamp', inplace= True)
df_monday.head()

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,3,dairy
2019-09-02 07:04:00,4,dairy
2019-09-02 07:04:00,5,spices


In [284]:
df_monday = df_monday.groupby(['customer_no']).resample('T').ffill()
df_monday.drop(['customer_no'], axis=1, inplace= True)
df_monday

location
customer_no timestamp                    
1           2019-09-02 07:03:00     dairy
            2019-09-02 07:04:00     dairy
            2019-09-02 07:05:00  checkout
2           2019-09-02 07:03:00     dairy
            2019-09-02 07:04:00     dairy
...                                   ...
1444        2019-09-02 21:48:00    spices
            2019-09-02 21:49:00  checkout
1445        2019-09-02 21:49:00     dairy
1446        2019-09-02 21:50:00     dairy
1447        2019-09-02 21:50:00     fruit

[10765 rows x 1 columns]

In [286]:
df_monday=df_monday.reset_index().set_index('timestamp')
df_monday.head()

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:04:00,1,dairy
2019-09-02 07:05:00,1,checkout
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,2,dairy


In [287]:
df_monday['prev_location']= df_monday['location'].shift(1)
df_monday.head()

,customer_no,location,prev_location
timestamp,,,
2019-09-02 07:03:00,1,dairy,NaN
2019-09-02 07:04:00,1,dairy,dairy
2019-09-02 07:05:00,1,checkout,dairy
2019-09-02 07:03:00,2,dairy,checkout
2019-09-02 07:04:00,2,dairy,dairy


In [288]:
# replace checkout with entrance

df_monday=df_monday[df_monday['prev_location'] != 'checkout']
df_monday.fillna('entrance' , inplace= True)
df_monday.head()

,customer_no,location,prev_location
timestamp,,,
2019-09-02 07:03:00,1,dairy,entrance
2019-09-02 07:04:00,1,dairy,dairy
2019-09-02 07:05:00,1,checkout,dairy
2019-09-02 07:04:00,2,dairy,dairy
2019-09-02 07:05:00,2,dairy,dairy


In [289]:
df_monday.groupby(['location','prev_location']).count()['customer_no'].unstack()

prev_location,dairy,drinks,entrance,fruit,spices
location,,,,,
checkout,310.0,426.0,NaN,524.0,177.0
dairy,2601.0,23.0,1.0,225.0,236.0
drinks,218.0,1250.0,NaN,132.0,217.0
fruit,181.0,186.0,NaN,1554.0,120.0
spices,185.0,162.0,NaN,122.0,478.0


In [290]:
pd.crosstab(df_monday['location'], df_monday['prev_location'], normalize = 0)

prev_location,dairy,drinks,entrance,fruit,spices
location,,,,,
checkout,0.215727,0.296451,0.000000,0.364649,0.123173
dairy,0.842839,0.007453,0.000324,0.072910,0.076474
drinks,0.119978,0.687947,0.000000,0.072647,0.119428
fruit,0.088682,0.091132,0.000000,0.761391,0.058795
spices,0.195354,0.171067,0.000000,0.128828,0.504752


In [304]:
# make it in order as the supermarket model

sup_mark = pd.crosstab(df_monday['location'], df_monday['prev_location'], normalize=0)
sup_mark = sup_mark[['entrance','fruit','spices','dairy','drinks']]

In [303]:
sup_mark.reindex(index = ['fruit','spices','dairy','drinks','checkout'])

prev_location,entrance,fruit,spices,dairy,drinks
location,,,,,
fruit,0.000000,0.761391,0.058795,0.088682,0.091132
spices,0.000000,0.128828,0.504752,0.195354,0.171067
dairy,0.000324,0.072910,0.076474,0.842839,0.007453
drinks,0.000000,0.072647,0.119428,0.119978,0.687947
checkout,0.000000,0.364649,0.123173,0.215727,0.296451
